# Road Following - Build TensorRT model for live demo

In this notebook, we will optimize the model we trained using TensorRT.

## Load the trained model

We will assume that you have already downloaded ``best_steering_model_xy.pth`` to work station as instructed in "train_model.ipynb" notebook. Now, you should upload model file to JetBot in to this notebook's directory. Once that's finished there should be a file named ``best_steering_model_xy.pth`` in this notebook's directory.

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

In [ ]:
import torchvision
import torch

#--- ResNet
model = torchvision.models.resnet18(pretrained=False)
# model = torchvision.models.resnet34(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

#--- MobileNet
# model = torchvision.models.mobilenet_v3_large(pretrained=False)
# model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, 2)

#--- Inception Net
# model = torchvision.models.inception_v3(pretrained=False)
# model.fc = torch.nn.Linear(model.fc.in_features, 2)
# if model.aux_logits:
#     model.AuxLogits.fc = torch.nn.Linear(model.AuxLogits.fc.in_features, 2)

model = model.cuda().eval().half()

Next, load the trained weights from the ``best_steering_model_xy.pth`` file that you uploaded.

In [ ]:
model.load_state_dict(torch.load('best_steering_model_xy_resnet18.pth'))
# model.load_state_dict(torch.load('best_steering_model_xy_resnet34.pth'))
# model.load_state_dict(torch.load('best_steering_model_xy_mobilenet_v3_large.pth'))
# model.load_state_dict(torch.load('best_steering_model_xy_inception_v3.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [ ]:
device = torch.device('cuda')

## TensorRT

> If your setup does not have `torch2trt` installed, you need to first install `torch2trt` by executing the following in the console.
```bash
cd $HOME
git clone https://github.com/NVIDIA-AI-IOT/torch2trt
cd torch2trt
sudo python3 setup.py install
```

Convert and optimize the model using torch2trt for faster inference with TensorRT. Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete.

In [ ]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()  # resnet
# data = torch.zeros((1, 3, 299, 299)).cuda().half()   # inception_v3

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [ ]:
torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_resnet18.pth')
# torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_resnet34.pth')
# torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_mobilenet_v3_large.pth')
# torch.save(model_trt.state_dict(), 'best_steering_model_xy_trt_inception_v3.pth')

## Next
Open live_demo_trt.ipynb to move JetBot with the TensorRT optimized model.